# Loading the data


In [1]:
%matplotlib inline

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics 
from sklearn.metrics import roc_curve,auc
from nltk.stem.porter import PorterStemmer



/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls '/content/drive/My Drive/amazon_data'

database.sqlite  Reviews.csv


In [0]:
# connection to the database
con = sqlite3.connect('/content/drive/My Drive/amazon_data/database.sqlite')

# filtering psitive and negative reviews
filtered_data = pd.read_sql_query("""
SELECT * FROM Reviews WHERE Score != 3
""",con)

In [5]:
filtered_data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
525809,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
525810,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
525811,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
525812,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [0]:
# seperate positive and negative reviews 
def partition(x):
    if x<3:
        return 'negative'
    return 'positive'


In [7]:
filtered_data.shape

(525814, 10)

In [0]:
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition)
filtered_data['Score'] = positiveNegative
# filtered_data['Score']

In [9]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


# Data cleaning: deduplication

In [10]:
display = pd.read_sql_query("""
SELECT * FROM Reviews WHERE Score != 3 AND UserId ="AR5J8UI46CURR"
ORDER BY ProductID
""",con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [0]:
# first sorting data according to productid in ascdending order
sorted_data = filtered_data.sort_values('ProductId',axis=0,ascending=True)


In [13]:
# remove duplicate entries 
final = sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"},keep='first',inplace=False)
final.shape

(364173, 10)

In [14]:
# Checking what % of data still remains 
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

69.25890143662969

In [15]:
# some data have helpfulnessdenominator less than helpfulnessnumerator
display = pd.read_sql_query("""
SELECT * FROM Reviews WHERE Score != 3 AND Id=44737 OR Id=64422
ORDER BY ProductID
""",con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [0]:
final = final[final.HelpfulnessNumerator <= final.HelpfulnessDenominator]

In [17]:
print(final.shape)

(364171, 10)


In [18]:
final['Score'].value_counts()

positive    307061
negative     57110
Name: Score, dtype: int64

# Bag of words

In [0]:
count_vect = CountVectorizer()
final_counts = count_vect.fit_transform(final['Text'].values)

In [20]:
type(final_counts)

scipy.sparse.csr.csr_matrix

In [21]:
final_counts.get_shape()

(364171, 115281)

# Text preprocessing (Stemming, stopword removal and lemmatization)

In [22]:
# find sentences containing HTML pages
import re
i =0 
for sent in final['Text'].values:
    if(len(re.findall('<.*?>',sent))):
        print(i)
        print(sent)
        break;
    i+=1;

6
I set aside at least an hour each day to read to my son (3 y/o). At this point, I consider myself a connoisseur of children's books and this is one of the best. Santa Clause put this under the tree. Since then, we've read it perpetually and he loves it.<br /><br />First, this book taught him the months of the year.<br /><br />Second, it's a pleasure to read. Well suited to 1.5 y/o old to 4+.<br /><br />Very few children's books are worth owning. Most should be borrowed from the library. This book, however, deserves a permanent spot on your shelf. Sendak's best.


In [23]:
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [24]:
stop = set(stopwords.words('english'))#set of stopwords
stop

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [25]:
# initializing a snowball stemmer
sno = nltk.stem.SnowballStemmer('english')
sno

In [0]:
# function to clean the word of any html tags
def cleanhtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr,' ',sentence)
    return cleantext

In [0]:
# clean word of any puctuations
def cleanpunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)    
    cleaned = re.sub(r'[.|,|)|(|\|/]',r'',cleaned)
    return cleaned

In [28]:
print(sno.stem('tasty'))

tasti


In [29]:
# code for implementing step by step checks
i=0
str1=' '
final_string=[]
all_positive_words=[]
all_negative_words=[]
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    for w in cleanhtml(sent).split():
        for cleaned_words in cleanpunc(w).split():
            print(cleaned_words)
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):
                s = (sno.stem(cleaned_words.lower())).encode('utf8')
                filtered_sentence.append(s)
                if(final['Score'].values)[i] == 'positive':
                    all_positive_words.append(s)
                if(final['Score'].values)[i] == 'negative':
                    all_negative_words.append(s)
            else:
                continue
        else:
            continue
    str1= b" ".join(filtered_sentence)
    final_string.append(str1)
    i+=1
        
        
        
        
        

Streaming output truncated to the last 5000 lines.
between
this
Cappuccino
drink
and
the
Starbucks
Latte
Primo
is
that
the
coffee
flavor
is
slightly
stronger
in
the
Starbucks
product
Also
with
the
Cappuccino
you
brew
the
coffee
first
and
then
use
the
milk
disc
With
the
Starbucks
Latte
Primo
you
brew
the
milk
first
then
the
coffee
Overall
I
think
this
is
a
bargain
buy
As
long
as
you
are
not
looking
for
a
slightly
stronger
coffee
flavor
Id
suggest
purchasing
this
instead
of
the
Starbucks
brand
as
both
products
use
expresso
coffee
and
milk
discs
I
love
coffee
Usually
I
would
not
recommend
Maxwell
House
as
I
consider
it
a
supermarket
brand
I
was
very
surprised
with
the
quality
of
this
cappuccino
My
wife
loves
it
too
Contrary
to
instructions
we
brew
milk
first
then
the
espresso
this
makes
it
look
very
nice
especially
on
a
transparent
cup
The
taste
is
smooth
and
velvety
with
low
acidity
and
aftertaste
You
can
buy
extra
espresso
T-Discs
separately
and
brew
an
extra
shot
if
you
really
like
you

In [0]:
final['CleanedText'] = final_string #adding column of cleaned text

In [31]:
final.head(3)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,b'this witti littl book make son laugh loud re...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",b'grew read these sendak book and watch the re...
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,b'this fun way for children learn their month ...


In [0]:
# store the table into a sqlite database table for future
conn = sqlite3.connect('final.sqlite')
c= conn.cursor()
conn.text_factory = str
final.to_sql('Reviews',conn,schema=None,if_exists='replace')

# bigrams ngrams 

In [35]:
freq_dist_positive=nltk.FreqDist(all_positive_words)
freq_dist_negative=nltk.FreqDist(all_negative_words)
print("Most common frequent words:",freq_dist_positive.most_common(20))
print("Most common negative words:",freq_dist_negative.most_common(20))

Most common frequent words: [(b'the', 940802), (b'and', 694915), (b'this', 356422), (b'for', 296936), (b'that', 223343), (b'have', 193922), (b'with', 193011), (b'you', 182915), (b'but', 177084), (b'are', 165798), (b'was', 146905), (b'not', 145989), (b'they', 141357), (b'like', 139072), (b'tast', 128077), (b'these', 118492), (b'good', 112017), (b'them', 108813), (b'flavor', 108653), (b'love', 107018)]
Most common negative words: [(b'the', 221255), (b'and', 121253), (b'this', 78387), (b'not', 54048), (b'that', 53227), (b'was', 51775), (b'for', 49496), (b'but', 42549), (b'have', 35928), (b'tast', 34300), (b'with', 33584), (b'like', 32225), (b'they', 31620), (b'you', 30690), (b'product', 28003), (b'are', 26639), (b'these', 20835), (b'one', 20521), (b'flavor', 19368), (b'had', 19302)]


In [0]:
# removing stop words like not should be avoided before building ngrams
count_vect = CountVectorizer(ngram_range=(1,2))
final_bigram_counts = count_vect.fit_transform(final['Text'].values)

In [38]:
final_bigram_counts.get_shape() 

(364171, 2910192)

# tf-idf

In [0]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(final['Text'].values)

In [46]:
final_tf_idf.get_shape()

(364171, 2910192)

In [47]:
features = tf_idf_vect.get_feature_names()
len(features)

2910192

In [48]:
features[100000:100010]

['ales until',
 'ales ve',
 'ales would',
 'ales you',
 'alessandra',
 'alessandra ambrosia',
 'alessi',
 'alessi added',
 'alessi also',
 'alessi and']

In [49]:
print(final_tf_idf[3,:].toarray()[0])

[0. 0. 0. ... 0. 0. 0.]


In [0]:
def top_tf_idf_feats(row,features,top_n=25):
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i],row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature','tfidf']
    return df

top_tfidf = top_tf_idf_feats(final_tf_idf[1,:].toarray()[0],features,25)

In [51]:
top_tfidf

,feature,tfidf
0,sendak books,0.173437
1,rosie movie,0.173437
2,paperbacks seem,0.173437
3,cover version,0.173437
4,these sendak,0.173437
5,the paperbacks,0.173437
6,pages open,0.173437
7,really rosie,0.168074
8,incorporates them,0.168074
9,paperbacks,0.168074


# word2vec


In [0]:
# using google word2vec
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

# model = KeyedVectors.load_word2vec_format('....')

In [0]:
# model.wv['computer']
# model.wv.similarity('woman','man')
# model.wv.most_similar('woman')

In [0]:
# train your own word2vec model
import gensim
i=0
list_of_sent=[]

for sent in final['Text'].values:
    filtered_sentence=[]
    sent = cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue
    list_of_sent.append(filtered_sentence)

In [55]:
print(final['Text'].values[0])
print("***************************************************************")
print(list_of_sent[0])

this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, India, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will STILL be able to recite from memory when he is  in college
***************************************************************
['this', 'witty', 'little', 'book', 'makes', 'my', 'son', 'laugh', 'at', 'loud', 'i', 'recite', 'it', 'in', 'the', 'car', 'as', 'were', 'driving', 'along', 'and', 'he', 'always', 'can', 'sing', 'the', 'refrain', 'hes', 'learned', 'about', 'whales', 'india', 'drooping', 'i', 'love', 'all', 'the', 'new', 'words', 'this', 'book', 'introduces', 'and', 'the', 'silliness', 'of', 'it', 'all', 'this', 'is', 'a', 'classic', 'book', 'i', 'am', 'willing', 'to', 'bet', 'my', 'son', 'will', 'still', 'be', 'able', 'to', 'recite', 'from', 'memory', 'when', 'he', 'is', 

In [0]:
w2v_model = gensim.models.Word2Vec(list_of_sent,min_count=5,size=50,workers=4)

In [57]:
words = list(w2v_model.wv.vocab)
print(len(words))

35274


In [58]:
w2v_model.wv.most_similar('tasty')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('tastey', 0.9147188067436218),
 ('yummy', 0.853003978729248),
 ('satisfying', 0.839580237865448),
 ('filling', 0.8250695466995239),
 ('delicious', 0.8156397938728333),
 ('flavorful', 0.7965638041496277),
 ('tasteful', 0.7794311046600342),
 ('nutritious', 0.7606377005577087),
 ('versatile', 0.7561256289482117),
 ('delish', 0.7503108382225037)]

In [59]:
count_vect_feat = count_vect.get_feature_names()
count_vect_feat.index('like')
print(count_vect_feat[64055])

activity great


# avg w2v, tfidf-w2v

In [60]:
# avg word2vec
sent_vectors = []
for sent in list_of_sent:
    sent_vec=np.zeros(50)
    cnt_words=0
    for word in sent:
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words+=1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
    
print(len(sent_vectors))
print(len(sent_vectors[0]))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


364171
50


In [0]:
# tf-idf weighted word2vec
tfidf_feat = tf_idf_vect.get_feature_names()

tfidf_sent_vectors = []
row=0
for sent in list_of_sent:
    sent_vec=np.zeros(50)
    weighted_sum=0
    for word in sent:
        try:
            vec = w2v_model.wv[word]
            tfidf = final_tf_idf[row,tfidf_feat.index(word)]
            sent_vec += (vec*tf_idf)
            weighted_sum += tf_idf
        except:
            pass
    sent_vec /= weighted_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide
